In [1]:
import numpy as np
import scipy.io as sio 

In [2]:
mat = sio.loadmat('ticelJudgments.mat')

# name of the 78 IPTs
names = mat['names'][0]

# groups indexes given by the 31 subjects. groups[1, 3]=groups[1, 2], then subject 1 grouped IPT 2 and 3 together 
groups = mat['ci']

In [3]:
ensemble = mat['ensemble'][0]

In [4]:
import pandas as pd
df = pd.read_csv("/homes/cv300/Documents/ipt-similarity/jasmp/seed_filelist.csv", index_col=0)
df['label'] = ensemble
df.groupby("instrument family")['label'].value_counts()

instrument family  label
Brass              10       6
                   3        5
                   13       5
                   11       2
                   14       2
PluckedStrings     8        5
                   5        2
                   9        1
Strings            18       7
                   9        5
                   19       5
                   5        4
                   15       4
                   6        2
                   7        2
                   8        2
                   16       2
                   12       1
                   17       1
Woodwind           17       4
                   1        3
                   2        2
                   4        2
                   3        1
                   12       1
                   14       1
                   15       1
Name: label, dtype: int64

In [5]:
sim = np.zeros((len(names), len(names)))
print(sim.shape)
for s in range(groups.shape[0]):
    for n in range(len(names)):
        for m in range(len(names)):
            if groups[s, n] == groups[s, m]:
                sim[n, m] += 1

(78, 78)


In [11]:
np.savez('ticelSimilarity.npz', names=names, groups=groups, similarity=sim)

In [6]:
bsim = np.where((sim / 31) > 0.5, 1, 0)
bsim.sum()

844

In [21]:
ensemble[np.where(bsim[20])]

array([10, 10, 13, 10, 10, 13, 13, 14, 13, 10, 14, 14], dtype=uint8)

In [ ]:
from ipt_sim.data import IptSimDataset
import os, tqdm
os.chdir("/homes/cv300/Documents/ipt-similarity/")

ds = IptSimDataset(feature="rand")

A = np.zeros((len(ds), len(ds)))

for i in tqdm.tqdm(range(A.shape[0])):
    i_seed = ds.filelist[i]['seed_id']
    for j in range(A.shape[1]):
        j_seed = ds.filelist[j]['seed_id']
        
        A[i, j] = bsim[i_seed, j_seed]
np.save("./jasmp/adjacency.mat", A)

Loading seed files ...


78it [00:00, 3736.32it/s]


Loading extended files ...


 63%|██████████████████████████████████████████████████▌                             | 4629/7319 [00:19<00:11, 242.72it/s]